<a href="https://colab.research.google.com/github/PFCEC/unitrade/blob/main/%E5%85%A7%E6%9C%9F%E8%A1%8C%E6%83%85%E7%AF%84%E4%BE%8B.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

事先準備

In [1]:
!pip install unitrade
#安裝顯示表格套件
!pip install plotly

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.4/13.4 MB 29.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 278.3/278.3 kB 13.8 MB/s eta 0:00:00


初始化

In [ ]:
import plotly.io as pio
pio.renderers.default = 'colab'  # 或嘗試 'notebook' 或 'iframe' 渲染器
from google.colab import output
output.enable_custom_widget_manager()

In [10]:
##API初始&登入
import pfcf
from pfcf.pfcf import *
from pfcf.product import *
from pfcf.quote.ddata import *
from typing import List
#匯入顯示表格套件
import plotly.graph_objs as go
from plotly.offline import init_notebook_mode
# 初始數據
init_notebook_mode(connected=True)
header1 = dict(values=['成交時間', '成交價','成交量','買價1','買量1' ,'買價2','買量2' ,'買價3','買量3' ,'買價4','買量4' ,'買價5','買量5',
             '賣價1','賣量1' ,'賣價2','賣量2' ,'賣價3','賣量3' ,'賣價4','賣量4' ,'賣價5','賣量5'])
data1 = [['','','','','','','','','','','','','','','','','','','','','','','']]
header2 = dict(values=['開盤時間','開盤價' ,'開盤量','最高低價時間' ,'最高','最低'])
data2 = [['','','','','','']]
header3 = dict(values=['現貨統計時間','現貨價格'])
data3 = [['','']]
# 創建初始表格
fig = go.FigureWidget(data=[go.Table( header=header1, cells=dict(values=list(zip(*data1)), align='left'),)])
fig.update_layout(height=100, margin=dict(t=0, b=0, l=0, r=0),autosize=True)
fig2 = go.FigureWidget(data=[go.Table( header=header2, cells=dict(values=list(zip(*data2)), align='left'),)])
fig2.update_layout(height=100, margin=dict(t=0, b=0, l=0, r=0),autosize=True)
fig3 = go.FigureWidget(data=[go.Table( header=header3, cells=dict(values=list(zip(*data3)), align='left'),)])
fig3.update_layout(height=100, margin=dict(t=0, b=0, l=0, r=0),autosize=True)
col_idx = {header: idx for idx, header in enumerate(header1['values'])}
col2_idx = {header: idx for idx, header in enumerate(header2['values'])}
col3_idx = {header: idx for idx, header in enumerate(header3['values'])}
display(fig)  # 顯示表格


FigureWidget({
    'data': [{'cells': {'align': 'left',
                        'values': [[''], [''], [''], [''], [''], [''], [''], [''],
                                   [''], [''], [''], [''], [''], [''], [''], [''],
                                   [''], [''], [''], [''], [''], [''], ['']]},
              'header': {'values': [成交時間, 成交價, 成交量, 買價1, 買量1, 買價2, 買量2, 買價3, 買量3,
                                    買價4, 買量4, 買價5, 買量5, 賣價1, 賣量1, 賣價2, 賣量2, 賣價3,
                                    賣量3, 賣價4, 賣量4, 賣價5, 賣量5]},
              'type': 'table',
              'uid': 'def9836c-326b-4f2e-97e5-3612366af825'}],
    'layout': {'autosize': True, 'height': 100, 'margin': {'b': 0, 'l': 0, 'r': 0, 't': 0}, 'template': '...'}
})

In [21]:
#初始API
api = Pfcf()
#錯誤事件處理
api.on_error = lambda error: print(f'api錯誤 {error}')
#系統登入
loginresponse = api.login("http://test167.pfctrade.com", "80009809769", "Aa123456", 'PSC_A226097195_20250310.pfx', '12345678')

#顯示登入結果
print(loginresponse)
print(f"登入旗標{api.login_status_flag}")
print(f"交易帳號{api.get_accounts()}")


The package version is 1.0.0.1
api錯誤 login -Timeout
LoginResponse(ok=False, error='Timeout')
登入旗標False
交易帳號None


查詢成交價

In [12]:
#查詢成交和最佳買賣價接收範例
productid='TXFK4'#商品代碼
if loginresponse.ok == True:
    display(fig)
    display(fig2)
     #查詢開盤價量
    tickdataopenResponse=api.dquote.query_tick_data_open(productid)
    if tickdataopenResponse.ok == True:
        tickdataopen=tickdataopenResponse.data
        cur = list(fig2.data[0].cells.values)
        cur[col2_idx['開盤時間']] = tickdataopen.opentime
        cur[col2_idx['開盤價']] = tickdataopen.openprice
        cur[col2_idx['開盤量'] ]= tickdataopen.openquantity
        with fig.batch_update():
            fig2.data[0].cells.values = cur
    else:
        print("tickdataopen "+tickdataopenResponse.error)
    #查詢成交價量
    tickDataTradeResponse=api.dquote.query_tick_data_trade(productid)
    if tickDataTradeResponse.ok == True:
        tickdatatrade=tickDataTradeResponse.data
        cur = list(fig.data[0].cells.values)
        cur[col_idx['成交時間']] = tickdatatrade.matchtime
        cur[col_idx['成交價']] = tickdatatrade.matchprice
        cur[col_idx['成交量']] = tickdatatrade.matchquantity
        with fig.batch_update():
            fig.data[0].cells.values = cur
    else:
        print("tickdatatrade "+tickDataTradeResponse.error)
    #查詢最高最低價
    tickDataHignLowResponse=api.dquote.query_tick_data_high_low(productid)
    if tickDataHignLowResponse.ok == True:
        tickdatahighlow=tickDataHignLowResponse.data
        cur = list(fig2.data[0].cells.values)
        cur[col2_idx['最高']] = tickdatahighlow.dayhighprice
        cur[col2_idx['最低']] = tickdatahighlow.daylowprice
        cur[col2_idx['最高低價時間']] = tickdatahighlow.showtime
        with fig.batch_update():
            fig2.data[0].cells.values = cur
    else:
        print("tickdatahighlow "+tickDataTradeResponse.error)
    #查詢最佳買賣價量
    tickDataBeforeBidOfferResponse=api.dquote.query_tick_data_bid_offer(productid)
    if tickDataBeforeBidOfferResponse.ok == True:
        tickdatabidoffer=tickDataBeforeBidOfferResponse.data
        cur = list(fig.data[0].cells.values)
        cur[col_idx['買價1']] = tickdatabidoffer.bp1
        cur[col_idx['買量1']] = tickdatabidoffer.bq1
        cur[col_idx['買價2']] = tickdatabidoffer.bp2
        cur[col_idx['買量2']] = tickdatabidoffer.bq2
        cur[col_idx['買價3']] = tickdatabidoffer.bp3
        cur[col_idx['買量3']] = tickdatabidoffer.bq3
        cur[col_idx['買價4']] = tickdatabidoffer.bp4
        cur[col_idx['買量4']] = tickdatabidoffer.bq4
        cur[col_idx['買價5']] = tickdatabidoffer.bp5
        cur[col_idx['買量5']] = tickdatabidoffer.bq5
        cur[col_idx['賣價1']] = tickdatabidoffer.sp1
        cur[col_idx['賣量1']] = tickdatabidoffer.sq1
        cur[col_idx['賣價2']] = tickdatabidoffer.sp2
        cur[col_idx['賣量2']] = tickdatabidoffer.sq2
        cur[col_idx['賣價3']] = tickdatabidoffer.sp3
        cur[col_idx['賣量3']] = tickdatabidoffer.sq3
        cur[col_idx['賣價4']] = tickdatabidoffer.sp4
        cur[col_idx['賣量4']] = tickdatabidoffer.sq4
        cur[col_idx['賣價5']] = tickdatabidoffer.sp5
        cur[col_idx['賣量5']] = tickdatabidoffer.sq5
        with fig.batch_update():
            fig.data[0].cells.values = cur
    else:
        print("tickdatabidoffer "+tickDataBeforeBidOfferResponse.error)




FigureWidget({
    'data': [{'cells': {'align': 'left',
                        'values': [[''], [''], [''], [''], [''], [''], [''], [''],
                                   [''], [''], [''], [''], [''], [''], [''], [''],
                                   [''], [''], [''], [''], [''], [''], ['']]},
              'header': {'values': [成交時間, 成交價, 成交量, 買價1, 買量1, 買價2, 買量2, 買價3, 買量3,
                                    買價4, 買量4, 買價5, 買量5, 賣價1, 賣量1, 賣價2, 賣量2, 賣價3,
                                    賣量3, 賣價4, 賣量4, 賣價5, 賣量5]},
              'type': 'table',
              'uid': 'def9836c-326b-4f2e-97e5-3612366af825'}],
    'layout': {'autosize': True, 'height': 100, 'margin': {'b': 0, 'l': 0, 'r': 0, 't': 0}, 'template': '...'}
})

FigureWidget({
    'data': [{'cells': {'align': 'left', 'values': [[''], [''], [''], [''], [''], ['']]},
              'header': {'values': ['開盤時間', '開盤價', '開盤量', '最高低價時間', '最高', '最低']},
              'type': 'table',
              'uid': 'de34e743-5647-4f39-bc3e-dd044c21c061'}],
    'layout': {'autosize': True, 'height': 100, 'margin': {'b': 0, 'l': 0, 'r': 0, 't': 0}, 'template': '...'}
})

In [13]:
#訂閱成交和最佳買賣價接收範例
productid='TXFK4'#商品代碼
#開盤價量接收處理
def on_tick_data_open(tickdataopen: DTickDataOpen):
    cur = list(fig2.data[0].cells.values)
    cur[col_idx['開盤時間']] = tickdataopen.opentime
    cur[col_idx['開盤價']] = tickdataopen.openprice
    cur[col_idx['開盤量'] ]= tickdataopen.openquantity
    with fig2.batch_update():
        fig2.data[0].cells.values = cur
#成交價量接收處理
def on_tick_data_trade(tickdatatrade: DTickDataTrade):
    cur = list(fig.data[0].cells.values)
    cur[col_idx['成交時間']] = tickdatatrade.matchtime
    cur[col_idx['成交價']] = tickdatatrade.matchprice
    cur[col_idx['成交量']] = tickdatatrade.matchquantity
    with fig.batch_update():
        fig.data[0].cells.values = cur
#最高最低價接收處理
def on_tick_data_high_low(tickdatahighlow: DTickDataHighLow):
    cur = list(fig2.data[0].cells.values)
    cur[col_idx['最高']] = tickdatahighlow.dayhighprice
    cur[col_idx['最低']] = tickdatahighlow.daylowprice
    cur[col_idx['最高低價時間']] = tickdatahighlow.showtime
    with fig2.batch_update():
        fig2.data[0].cells.values = cur
#最佳買賣價量接收處理
def on_tick_data_bid_offer(tickdatabidoffer: DTickDataBidOffer):
    cur = list(fig.data[0].cells.values)
    cur[col_idx['買價1']] = tickdatabidoffer.bp1
    cur[col_idx['買量1']] = tickdatabidoffer.bq1
    cur[col_idx['買價2']] = tickdatabidoffer.bp2
    cur[col_idx['買量2']] = tickdatabidoffer.bq2
    cur[col_idx['買價3']] = tickdatabidoffer.bp3
    cur[col_idx['買量3']] = tickdatabidoffer.bq3
    cur[col_idx['買價4']] = tickdatabidoffer.bp4
    cur[col_idx['買量4']] = tickdatabidoffer.bq4
    cur[col_idx['買價5']] = tickdatabidoffer.bp5
    cur[col_idx['買量5']] = tickdatabidoffer.bq5
    cur[col_idx['賣價1']] = tickdatabidoffer.sp1
    cur[col_idx['賣量1']] = tickdatabidoffer.sq1
    cur[col_idx['賣價2']] = tickdatabidoffer.sp2
    cur[col_idx['賣量2']] = tickdatabidoffer.sq2
    cur[col_idx['賣價3']] = tickdatabidoffer.sp3
    cur[col_idx['賣量3']] = tickdatabidoffer.sq3
    cur[col_idx['賣價4']] = tickdatabidoffer.sp4
    cur[col_idx['賣量4']] = tickdatabidoffer.sq4
    cur[col_idx['賣價5']] = tickdatabidoffer.sp5
    cur[col_idx['賣量5']] = tickdatabidoffer.sq5
    with fig.batch_update():
        fig.data[0].cells.values = cur
#訂閱接收事件
api.dquote.on_tick_data_open = on_tick_data_open
api.dquote.on_tick_data_high_low = on_tick_data_high_low
api.dquote.on_tick_data_trade = on_tick_data_trade
api.dquote.on_tick_data_bid_offer = on_tick_data_bid_offer
if loginresponse.ok == True:
    subok,msg=api.dquote.subscribe_trade_bid_offer(productid)
    print(f'註冊成交買賣價量 {productid} {subok} {msg}')
    subok,msg=api.dquote.subscribe_high_low(productid)
    print(f'註冊最高最低價 {productid} {subok} {msg}')
    subok,msg=api.dquote.subscribe_open(productid)
    print(f'註冊開收盤價 {productid} {subok} {msg}')
    display(fig)
    display(fig2)
else:
    print(loginresponse.error)

註冊成交買賣價量 TXFK4 True 
註冊最高最低價 TXFK4 True 
註冊開收盤價 TXFK4 True 


FigureWidget({
    'data': [{'cells': {'align': 'left',
                        'values': [105042087000, 23392.0, 2, 23390.0, 5, 23389.0,
                                   8, 23388.0, 10, 23387.0, 10, 23386.0, 6,
                                   23392.0, 2, 23393.0, 16, 23394.0, 4, 23395.0, 7,
                                   23396.0, 9]},
              'header': {'values': [成交時間, 成交價, 成交量, 買價1, 買量1, 買價2, 買量2, 買價3, 買量3,
                                    買價4, 買量4, 買價5, 買量5, 賣價1, 賣量1, 賣價2, 賣量2, 賣價3,
                                    賣量3, 賣價4, 賣量4, 賣價5, 賣量5]},
              'type': 'table',
              'uid': 'def9836c-326b-4f2e-97e5-3612366af825'}],
    'layout': {'autosize': True, 'template': '...'}
})

FigureWidget({
    'data': [{'cells': {'align': 'left', 'values': ['084500048000', 23555.0, 212, '100258374000', 23618.0, 23309.0]},
              'header': {'values': ['開盤時間', '開盤價', '開盤量', '最高低價時間', '最高', '最低']},
              'type': 'table',
              'uid': 'de34e743-5647-4f39-bc3e-dd044c21c061'}],
    'layout': {'autosize': True, 'template': '...'}
})

In [14]:
#反註冊報價
unsub_ok,msg=api.dquote.unsubscribe_trade_bid_offer(productid)
print(f'反註冊成交買賣價量 {productid} {unsub_ok} {msg}')
unsub_ok,msg=api.dquote.unsubscribe_high_low(productid)
print(f'反註冊最高最低價 {productid} {unsub_ok} {msg}')
unsub_ok,msg=api.dquote.unsubscribe_open(productid)
print(f'反註冊開收盤價 {productid} {unsub_ok} {msg}')

反註冊成交買賣價量 TXFK4 True 
反註冊最高最低價 TXFK4 True 
反註冊開收盤價 TXFK4 True 


In [15]:
#查詢現貨價量範例
index_id='TXF'#代碼
display(fig3)
if loginresponse.ok == True:
     #查詢
    indexdataResponse=api.dquote.query_index_data(index_id)
    if indexdataResponse.ok == True:
        indexdata=indexdataResponse.data
        cur = list(fig3.data[0].cells.values)
        cur[col3_idx['現貨統計時間']] = indexdata.index_time
        cur[col3_idx['現貨價格']] = indexdata.index_value
        with fig3.batch_update():
            fig3.data[0].cells.values = cur
    else:
        print(indexdataResponse.error)

FigureWidget({
    'data': [{'cells': {'align': 'left', 'values': [[''], ['']]},
              'header': {'values': ['現貨統計時間', '現貨價格']},
              'type': 'table',
              'uid': '7e74a290-b25d-4f92-a72b-161ecc6dec3f'}],
    'layout': {'autosize': True, 'height': 100, 'margin': {'b': 0, 'l': 0, 'r': 0, 't': 0}, 'template': '...'}
})

In [16]:
#訂閱現貨價量接收範例
index_id='TXF' #代碼
#接收現貨價量處理
def on_index_data(indexdata: DIndexData):
    cur = list(fig3.data[0].cells.values)
    cur[col3_idx['現貨統計時間']] = indexdata.index_time
    cur[col3_idx['現貨價格']] = indexdata.index_value
    with fig3.batch_update():
        fig3.data[0].cells.values = cur
#訂閱事件
api.dquote.on_index_data = on_index_data
if loginresponse.ok == True:
    #註冊報價
    subok,msg=api.dquote.subscribe_index_data('1',index_id)
    print(f'註冊現貨價量 {index_id} {subok} {msg}')
    display(fig3)
else:
    print(loginresponse.error)

註冊現貨價量 TXF True 


FigureWidget({
    'data': [{'cells': {'align': 'left', 'values': ['105120000000', 23383.69]},
              'header': {'values': ['現貨統計時間', '現貨價格']},
              'type': 'table',
              'uid': '7e74a290-b25d-4f92-a72b-161ecc6dec3f'}],
    'layout': {'autosize': True, 'template': '...'}
})

In [17]:
#反註冊現貨價量接收
unsub_ok,msg=api.dquote.unsubscribe_index_data('1',index_id)
print(f'反註冊現貨價量 {index_id} {unsub_ok} {msg}')

反註冊現貨價量 TXF True 


In [20]:
#系統登出
api.logout()
print('登出完畢')
print(f"登入旗標{api.login_status_flag}")


登出完畢
登入旗標False
